Set `autoreload` to execute the change in `.py` files.


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import _pickle
from pathlib import Path
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

from __utils__ import images_processing
from __utils__ import labels_processing
from __utils__ import labeling
from __utils__ import loso_preparing
from __utils__ import functions

In [ ]:
dataset_dir = "D:/Databases/CAS(ME)^2"
# dataset_dir = "I:/HEH/Databases/CAS(ME)^2"
# dataset_dir = "/data/disk1/heh/databases/CAS(ME)^2"

images_loading = False
image_size = 128
load_cropped_images = False
expression_type = "mae"  # macro-expression spotting
# expression_type = "me"  # micro-expression spotting
debug_preds = True
labeling_function = "pseudo_labeling"
# labeling_function = "original_labeling"
model_names = {
    0: "SOFTNet",
    1: "SOFTNetCBAM",
    2: "ViT-B",
    3: "SL-ViT-B",
    4: "Swin-T",
    5: "Swin-S",
    6: "L-Swin-T",
    7: "S-Swin-T",
    8: "SL-Swin-T",
    9: "SL-Swin-S",
}
model_name = model_names[8]
batch_size = 48
epochs = 25
save_preds = False
preds_stem = (
    f"{expression_type}_"
    + model_name.lower().replace("-", "_")
    + f"_batch_size_{batch_size}"
    + f"_epochs_{epochs}"
    + f"_{labeling_function}"
    + f"_{image_size}"
    + "_3"
)
preds_path = Path(dataset_dir, "preds", preds_stem).with_suffix(".pkl")
print(f"preds_path: {preds_path}")

## Load Images


When debug the image processing, the videos_images is from cropped_rawpic, whereas the other variables are from rawpic.


In [4]:
videos_images, subjects, subjects_videos_code = images_processing.load_images_dev(
    dataset_dir,
    images_loading=images_loading,
    image_size=image_size,
    load_cropped_images=load_cropped_images,
)

subject:  s15
subject:  s16
subject:  s19
subject:  s20
subject:  s21
subject:  s22
subject:  s23
subject:  s24
subject:  s25
subject:  s26
subject:  s27
subject:  s29
subject:  s30
subject:  s31
subject:  s32
subject:  s33
subject:  s34
subject:  s35
subject:  s36
subject:  s37
subject:  s38
subject:  s40


In [5]:
print("subjects:", subjects)
print("subjects_videos_code:", subjects_videos_code)

subjects: ['s15', 's16', 's19', 's20', 's21', 's22', 's23', 's24', 's25', 's26', 's27', 's29', 's30', 's31', 's32', 's33', 's34', 's35', 's36', 's37', 's38', 's40']
subjects_videos_code: [['0101', '0102', '0401', '0402', '0502', '0503', '0505', '0508'], ['0101', '0102', '0401', '0402', '0502', '0505', '0507'], ['0102', '0402', '0505', '0507', '0502'], ['0502'], ['0101', '0401'], ['0101', '0102', '0402', '0503', '0508'], ['0102', '0402', '0503', '0507'], ['0101', '0401', '0402', '0502', '0507'], ['0101', '0102', '0502', '0508'], ['0101', '0102', '0401', '0503'], ['0101', '0102', '0401', '0402', '0502', '0503', '0505', '0507', '0508'], ['0502'], ['0101', '0102', '0401', '0502', '0503', '0505', '0507'], ['0101', '0401', '0402', '0502', '0503', '0505', '0507'], ['0101', '0102', '0401', '0402', '0502', '0503', '0505', '0507', '0508'], ['0102', '0402'], ['0401', '0402', '0503'], ['0102'], ['0401', '0505'], ['0101', '0402', '0502', '0505', '0507', '0508'], ['0502', '0507'], ['0401', '0502', '

## Load Excel 


In [7]:
Excel_data = labels_processing.load_excel(dataset_dir)
Excel_data.head(5)

,participant,video_name_&_expression_number,onset,apex,offset,AUs,extimated_emotion,expression_type,self-reported_emotion,video_name,video_code,subject
0,1,anger1_1,557,572,608,4+10+14+15,negative,macro-expression,anger,anger1,0401,s15
1,1,anger1_2,2854,2862,2871,38,others,macro-expression,sadness,anger1,0401,s15
2,1,anger2_1,2155,2163,0,NaN,negative,macro-expression,anger,anger2,0402,s15
3,1,anger2_2,3363,3371,3383,4+7+14,negative,macro-expression,anger,anger2,0402,s15
4,1,anger2_3,3380,3386,3407,4+14+38,negative,macro-expression,anger,anger2,0402,s15


## Load Ground Truth Labels


In [8]:
(
    clean_videos_images,
    clean_subjects_videos_code,
    clean_subjects,
    clean_subjects_videos_ground_truth_labels,
) = labels_processing.load_ground_truth_labels(
    dataset_dir,
    expression_type,
    videos_images,
    subjects_videos_code,
    subjects,
    Excel_data,
)

required_videos_index:  [0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 78, 79, 80, 81, 82, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96]
len(clean_videos_images) = 88


In [9]:
print("len(clean_subjects): ", len(clean_subjects))
print("clean_subjects: ", clean_subjects)
print("len(clean_subjects_videos_code): ", len(clean_subjects_videos_code))
print("clean_subjects_videos_codes: ", clean_subjects_videos_code)
print(
    "len(clean_subjects_videos_ground_truth_labels): ",
    len(clean_subjects_videos_ground_truth_labels),
)

# 7 (s23) has happy1 (0502) in excel but the folder name is happy2 (0503)
print(
    "clean_subjects_videos_ground_truth_labels[6]: ",
    clean_subjects_videos_ground_truth_labels[6],
)

len(clean_subjects):  20
clean_subjects:  ['s15' 's16' 's19' 's20' 's21' 's22' 's23' 's24' 's25' 's26' 's27' 's30'
 's31' 's32' 's33' 's34' 's36' 's37' 's38' 's40']
len(clean_subjects_videos_code):  20
clean_subjects_videos_codes:  [['0101', '0102', '0401', '0402', '0502', '0503', '0505'], ['0101', '0102', '0401', '0402', '0502', '0505', '0507'], ['0102', '0505', '0507'], ['0502'], ['0101', '0401'], ['0101', '0102', '0402', '0503', '0508'], ['0102', '0402', '0507'], ['0101', '0401', '0402', '0502', '0507'], ['0102', '0502', '0508'], ['0101', '0102', '0401', '0503'], ['0101', '0102', '0401', '0402', '0502', '0503', '0505', '0507', '0508'], ['0102', '0401', '0502', '0503', '0505', '0507'], ['0101', '0401', '0402', '0502', '0503', '0505', '0507'], ['0101', '0102', '0401', '0402', '0502', '0503', '0505', '0507'], ['0102', '0402'], ['0401', '0402', '0503'], ['0401', '0505'], ['0101', '0402', '0502', '0505', '0507', '0508'], ['0502', '0507'], ['0401', '0502', '0503']]
len(clean_subjects_vide

In [10]:
total_len = 0
for index, clean_subject_videos_code in enumerate(clean_subjects_videos_code):
    ground_truth_len = 0
    for i in clean_subjects_videos_ground_truth_labels[index]:
        for j in i:
            ground_truth_len += 1
    print(
        f"{index} {clean_subjects[index]}: {clean_subject_videos_code}, ground truth len: {ground_truth_len}"
    )
    total_len += ground_truth_len
print("total len: ", total_len)

0 s15: ['0101', '0102', '0401', '0402', '0502', '0503', '0505'], ground truth len: 15
1 s16: ['0101', '0102', '0401', '0402', '0502', '0505', '0507'], ground truth len: 46
2 s19: ['0102', '0505', '0507'], ground truth len: 4
3 s20: ['0502'], ground truth len: 3
4 s21: ['0101', '0401'], ground truth len: 2
5 s22: ['0101', '0102', '0402', '0503', '0508'], ground truth len: 16
6 s23: ['0102', '0402', '0507'], ground truth len: 6
7 s24: ['0101', '0401', '0402', '0502', '0507'], ground truth len: 24
8 s25: ['0102', '0502', '0508'], ground truth len: 5
9 s26: ['0101', '0102', '0401', '0503'], ground truth len: 6
10 s27: ['0101', '0102', '0401', '0402', '0502', '0503', '0505', '0507', '0508'], ground truth len: 32
11 s30: ['0102', '0401', '0502', '0503', '0505', '0507'], ground truth len: 17
12 s31: ['0101', '0401', '0402', '0502', '0503', '0505', '0507'], ground truth len: 26
13 s32: ['0101', '0102', '0401', '0402', '0502', '0503', '0505', '0507'], ground truth len: 48
14 s33: ['0102', '0402

## Calculate k


In [11]:
k = labels_processing.calculate_k(clean_subjects_videos_ground_truth_labels)

k (Half of average length of expression) =  18


## Labeling


In [18]:
if debug_preds is False:
    if labeling_function == "pseudo_labeling":
        labels = labeling.get_pseudo_labels(
            clean_videos_images, clean_subjects_videos_ground_truth_labels, k
        )
    elif labeling_function == "original_labeling":
        labels = labeling.get_original_labels(
            clean_videos_images, clean_subjects_videos_ground_truth_labels, k
        )

Total frames: 231999


## Prepare for LOSO


In [19]:
y, groups = loso_preparing.prepare_for_loso(
    labels,
    clean_subjects,
    clean_videos_images,
    clean_subjects_videos_ground_truth_labels,
    k,
)

Frame Index for each subject:-

subject s15 ( group = 0): 0 -> 18470
subject s15 has 7 clean video(s)
sum clean_subject_videos_ground_truth_labels_len:  7

subject s16 ( group = 1): 18470 -> 37392
subject s16 has 7 clean video(s)
sum clean_subject_videos_ground_truth_labels_len:  14

subject s19 ( group = 2): 37392 -> 43977
subject s19 has 3 clean video(s)
sum clean_subject_videos_ground_truth_labels_len:  17

subject s20 ( group = 3): 43977 -> 46233
subject s20 has 1 clean video(s)
sum clean_subject_videos_ground_truth_labels_len:  18

subject s21 ( group = 4): 46233 -> 51966
subject s21 has 2 clean video(s)
sum clean_subject_videos_ground_truth_labels_len:  20

subject s22 ( group = 5): 51966 -> 62840
subject s22 has 5 clean video(s)
sum clean_subject_videos_ground_truth_labels_len:  25

subject s23 ( group = 6): 62840 -> 71496
subject s23 has 3 clean video(s)
sum clean_subject_videos_ground_truth_labels_len:  28

subject s24 ( group = 7): 71496 -> 87084
subject s24 has 5 clean video

## Training


In [24]:
if debug_preds is False:
    from __utils__.training_dev import train

    preds = train(
        dataset_dir,
        clean_subjects,
        y=y,
        expression_type=expression_type,
        model_name=model_name,
        train_or_not=True,
        epochs=epochs,
        batch_size=batch_size,
    )
else:
    with open(preds_path, "rb") as pkl_file:
        preds = _pickle.load(pkl_file)
        pkl_file.close()

In [ ]:
if save_preds is True:
    with open(preds_path, "wb") as pkl_file:
        _pickle.dump(preds, pkl_file)
        pkl_file.close()

## Spotting and Evaluation


In [25]:
metric_fn, result_dict = functions.spot_and_evaluate(
    preds,
    clean_subjects_videos_ground_truth_labels,
    clean_videos_images,
    clean_subjects,
    clean_subjects_videos_code,
    k,
    p=0.60,
    show_plot_or_not=False,
)

Split 0 / 20 is in process.
0 video(s) have been processed.
The current video be processed: subject s15, video 0101
The current video be processed: subject s15, video 0102
The current video be processed: subject s15, video 0401
The current video be processed: subject s15, video 0402
The current video be processed: subject s15, video 0502
The current video be processed: subject s15, video 0503
The current video be processed: subject s15, video 0505

True Positive: 3 False Posive: 22 False Negative: 12
Precision = 0.12000000000000022, Recall =0.20000000000000023, F1-Score = 0.15000000000000044
Split 0 / 20 is processed.

Split 1 / 20 is in process.
7 video(s) have been processed.
The current video be processed: subject s16, video 0101
The current video be processed: subject s16, video 0102
The current video be processed: subject s16, video 0401
The current video be processed: subject s16, video 0402
The current video be processed: subject s16, video 0502
The current video be processed: s

## Final Evaluation


In [26]:
functions.final_evaluate(metric_fn, result_dict)

True Positive: 68 False Posive: 297 False Negative: 230
COCO AP@[.5:.95]: 0.0151
Final Precision = 0.18630136986301393,
Final Recall =0.22818791946308747,
Final F1-Score = 0.2051282051282056

Highest Precision = 0.21951219512195144,
Highest Recall =0.2520000000000002,
Highest F1-Score = 0.23463687150838033


| Parameters | Value | Value | Value | Value
| --- | --- | --- | --- | ---
| model | 3D-CNN | SOFTNet | SOFTNet (generator) | SOFTNet (dev)
| epochs | | 10 | 20 | 20 (252 m)
| batch_size | | 48 | 48 | 48
| learning_rate | | 0.0005 | 0.0005 | 0.0005
| True Positive | | 90 | 95 | 83
| False Positive | | 357 | 357 | 316
| False Negative | | 210 | 203 | 215
| Precision | | 0.0213 | 0.2102 | 0.2080
| Recall | | 0.3000 | 0.3188 | 0.2785
| F1-Score | 0.2145 | 0.2410 | 0.2533 | 0.2381


| Parameters | Value | Value
| --- | --- | ---
| model | ViT (generator) | Vit (dev)
| epochs | 20 | 20
| batch_size | 48 | 96
| learning_rate | 0.0005 | 0.0005
| True Positive | 57 | 50
| False Positive | 520 | 519
| False Negative | 241 | 248
| Precision | 0.0987 | 0.0878
| Recall | 0.1912 | 0.1677
| F1-Score | 0.1302 | 0.1153


| Parameters | Value |
| --- | --- | 
| model | SL-ViT |
| epochs | 20 (721 m) |
| batch_size | 96 |
| learning_rate | 0.0005 |
| True Positive | 48 |
| False Positive | 269 |
| False Negative | 250 |
| Precision | 0.1514 |
| Recall | 0.1610 |
| F1-Score | 0.1560 |


| Parameters | Value | Value | Value | Value
| --- | --- | --- | --- | ---
| model | SL-Swin-S | SL-Swin-S | SL-Swin-S | SL-Swin-S
| epochs | 25 (6495 m) | 20 (3784 m) | 25 (4682) | 20 (2690 m)
| batch_size | 32 | 48 | 48 | 96
| learning_rate | 0.0005 | 0.0005 | 0.0005 | 0.0005
| True Positive | 69 | 66 | 69 | 68
| False Positive | 281 | 271 | 262 | 297
| False Negative | 229 | 232 | 229 | 230
| Precision | 0.1971 | 0.1958 | 0.2084 | 0.1863
| Recall | 0.2315 | 0.2214 | 0.2315 | 0.2281
| F1-Score | 0.2129 | 0.2078 | 0.2194 | 0.2051


## Ablation Study


In [27]:
ablation_dict = functions.ablation_study_p_dev(
    preds,
    clean_subjects_videos_ground_truth_labels,
    clean_videos_images,
    clean_subjects,
    clean_subjects_videos_code,
    k,
)

 p | TP | FP | FN | Precision | Recall | F1-Score
0.05 | 163 | 2593 | 135 | 0.0591 | 0.5470 | 0.1067 |
0.10 | 149 | 1842 | 149 | 0.0748 | 0.5000 | 0.1302 |
0.15 | 138 | 1387 | 160 | 0.0905 | 0.4631 | 0.1514 |
0.20 | 125 | 1079 | 173 | 0.1038 | 0.4195 | 0.1664 |
0.25 | 114 | 854 | 184 | 0.1178 | 0.3826 | 0.1801 |
0.30 | 108 | 703 | 190 | 0.1332 | 0.3624 | 0.1948 |
0.35 | 96 | 590 | 202 | 0.1399 | 0.3221 | 0.1951 |
0.40 | 89 | 488 | 209 | 0.1542 | 0.2987 | 0.2034 |
0.45 | 81 | 413 | 217 | 0.1640 | 0.2718 | 0.2045 |
0.50 | 71 | 342 | 227 | 0.1719 | 0.2383 | 0.1997 |
0.55 | 68 | 297 | 230 | 0.1863 | 0.2282 | 0.2051 |
0.60 | 60 | 265 | 238 | 0.1846 | 0.2013 | 0.1926 |
0.65 | 47 | 214 | 251 | 0.1801 | 0.1577 | 0.1682 |
0.70 | 36 | 177 | 262 | 0.1690 | 0.1208 | 0.1409 |
0.75 | 32 | 147 | 266 | 0.1788 | 0.1074 | 0.1342 |
0.80 | 31 | 122 | 267 | 0.2026 | 0.1040 | 0.1375 |
0.85 | 26 | 104 | 272 | 0.2000 | 0.0872 | 0.1215 |
0.90 | 21 | 88 | 277 | 0.1927 | 0.0705 | 0.1032 |
0.95 | 20 | 83 | 278 | 